In [1]:
import BrownsvilleAPI
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Brownsville Analysis

This analysis uses the Brownsville API to perform a prelimanry analysis on the dataset. It explores the most common types of complaints over the years and identifies trends in the number of complaints reported over the years.

In [2]:
# brownsville = BrownsvilleAPI.Brownsville(force_load=True)
brownsville = BrownsvilleAPI.Brownsville()

Loading cached dataset...


In [ ]:
columns = ["buildingid", "address", "latitude", "longitude"]
unique_addresses = brownsville.data[columns].groupby(columns).size()
unique_addresses.index.get_level_values(0).to_series().shape

In [3]:
brownsville.display_map()

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
float64 float64 float64
(79550,) (79550,) (79550,)


TypeError: Object of type int64 is not JSON serializable

In [ ]:
complaints_per_building = brownsville.complaints
# complaints_per_building = brownsville.complaints
wf = lambda b_id: complaints_per_building[b_id]
weights = brownsville.data["buildingid"].to_frame().apply(wf).astype("Float64").values
# weight = (weight-weight.mean())/weight.std()
weights = ((weights-min(weights))/(max(weights)-min(weights)))
weights
# print(brownsville.data["buildingid"].shape[0], len(weight))


In [ ]:
brownsville.data.head()

In [ ]:
brownsville.data.columns

## Building Coverage

In [ ]:
print(f"Building coverage: {len(brownsville.buildings)}")

## Complaint types

In [ ]:
building_common_complaints = []
for _id in brownsville.buildings:
    common_complaints = brownsville.get_feature_occurrences_by_building(
                                                            _id,
                                                            by=["majorcategory", "minorcategory"],
                                                            find_all=True)

    building_common_complaints.append((_id, common_complaints))
building_common_complaints.sort(key=lambda e: e[1].values.sum(), reverse=True)

In [ ]:
for building in building_common_complaints[:10]:
    id_, complaints = building
    major_category, minor_category = complaints.index[0]
    num_complaints = complaints.values.sum()

    print("Building ID:", id_)
    print("Most common major category:", major_category)
    print("Most common minor category:", minor_category)
    print("Number of complaints:", num_complaints)

    print()

## Complaints over time

In [ ]:
complaints_by_month = brownsville.records_by_date(period="year")
x, y = complaints_by_month.index, complaints_by_month.values
plt.bar(x, y)

In [ ]:
x, y = brownsville.records_by_season()
plt.bar(x, y)
plt.title("Number of complaints by season")
plt.show()

In [ ]:
complaints_by_month = brownsville.records_by_date()
x, y = complaints_by_month.index, complaints_by_month.values
plt.bar(x, y)
plt.title("Number of complaints by month")
plt.show()

In [ ]:
steps = 4
years = brownsville.records_by_date(period="year", num_years=8, step=steps)

for year in years:

    x = list(year.index.values)
    y = list(year.values)
    label = f"{year.index[0]} to {year.index[-1]}"

    plt.plot(np.arange(0, steps), y, label=label)
    
plt.xlabel("Number of years")
plt.ylabel("Complaints reported")
plt.title("Number of complaints over a period of 8 years on 4 year intervals")
plt.legend()
plt.show()

In [ ]:
brownsville.get_date_range("received")

In [ ]:
len(brownsville.data["address"].unique())

In [ ]:
brownsville.display_map(save_map=True)

In [ ]:
no_complaints_filter = (brownsville.data["majorcategoryid"].isna()) & (
    brownsville.data["minorcategoryid"].isna()
)
brownsville.data[(no_complaints_filter)].isna().sum().value_counts

In [ ]:
brownsville.data.shape